In [15]:
%load_ext autoreload
%autoreload 2

from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign, SymmetricTransmonBuilder
from adjoint_sim_sf import AdjointEvaluator
import numpy as np
import matplotlib.pyplot as plt

params = np.array([0.20,0.25])

std = SymmetricTransmonDesign()
std.get_interior_area(params)
fig = std.show_polygons(params)

fig.show()
print(std.get_interior_area(params))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.17561198314241452


In [ ]:

import numpy as np
from datetime import datetime

from SQDMetal.COMSOL.Model import COMSOL_Model
from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign
from adjoint_sim_sf.AdjointSolver import AdjointEvaluator
from adjoint_sim_sf.Optimiser import Optimiser
from adjoint_sim_sf import Experiment

# 1) COMSOL engine
if COMSOL_Model._engine is None:
    COMSOL_Model.init_engine()

# 2) Evaluator config (must match AdjointEvaluator.update_params / attributes)
base_config = {
    "freq_value": 8e9,
    "num_adj_sample_points": 20,
    "param_perturbation": [1e-5],
    "fwd_source_strength": 1e-2,
    "adjoint_rotation": float(np.pi / 2),
}

params = np.array([0.19, 0.20])
designer  = SymmetricTransmonDesign()
evaluator = AdjointEvaluator(designer, config=base_config)

evaluator.evaluate(params, 0.01)



jj: [Source(location=array([0.    , 0.    , 0.0001]), direction=array([0, 1, 0]), strength=None)]
Saved  20251016-718896.mph
Saved  20251016-794849.mph
Params: [0.19 0.2 ],Loss: -71104908064794.33, ||grad||: 3.013131e+16, grad: [-2.13042409e+16 -2.13078717e+16]


(array([-2.13042409e+16, -2.13078717e+16]), -71104908064794.33)

In [26]:
evaluator.parametric_designer.get_interior_area(params)

0.1339572240871911

In [29]:
from shapely.geometry import MultiPolygon, Polygon
import numpy as np
from adjoint_sim_sf.ParametricDesign import  SymmetricTransmonPolygonConstructor

mp = SymmetricTransmonPolygonConstructor().make_polygons(params)

# Per-polygon areas (mm²) and in m²
areas_mm2 = [p.area for p in (mp.geoms if isinstance(mp, MultiPolygon) else [mp])]
areas_m2  = [a * 1e-6 for a in areas_mm2]
total_mm2 = sum(areas_mm2)
total_m2  = total_mm2 * 1e-6

print("Per-pad areas (mm²):", areas_mm2)
print("Per-pad areas (m²): ", areas_m2)
print("Total area (mm²):   ", total_mm2)
print("Total area (m²):    ", total_m2)

# Sanity checks
assert np.isclose(total_mm2, mp.area)                  # MultiPolygon sums its parts
assert np.isclose(total_m2, mp.area * 1e-6)            # mm² → m²


Per-pad areas (mm²): [0.06697861204359555, 0.06697861204359555]
Per-pad areas (m²):  [6.697861204359554e-08, 6.697861204359554e-08]
Total area (mm²):    0.1339572240871911
Total area (m²):     1.339572240871911e-07
